We classify all census knots that share the same 0-surgery. (Here the 0-surgery is measured with respect to the Seifert framing.)

In [1]:
import snappy
import time
import csv

First, we sort the knots into groups with the same Alexander polynomials.

In [2]:
start_time = time.time()
ALEX_GROUP=[]
alexander_polynomials=[]

for K in snappy.CensusKnots():
    f=K.alexander_polynomial()
    if f not in alexander_polynomials:
        alexander_polynomials.append(f)
        ALEX_GROUP.append([f,[K.identify()[0].name()]])
    else:
        for [pol,knots] in ALEX_GROUP:
            if pol==f:
                knots.append(K.identify()[0].name())

print('Number of different Alexander polynomials:',len(alexander_polynomials))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

Number of different Alexander polynomials: 1135
Time taken: 1.7812053283055624 minutes 


In [3]:
ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT=[x for x in ALEX_GROUP if len(x[1])>1]

In [4]:
len(ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT)

74

In [5]:
ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT

[[a^2 - 3*a + 1, ['m004', 't11898']],
 [2*a^2 - 3*a + 2, ['m015', 't10416']],
 [2*a^2 - 5*a + 2, ['m032', 'm372', 's879', 'v2861', 't08306', 'o9_22066']],
 [3*a^2 - 5*a + 3, ['m053', 'v3169']],
 [4*a^2 - 7*a + 4, ['m094', 's648', 'v2001', 'v2743']],
 [a^4 - 2*a^3 + a^2 - 2*a + 1, ['m199', 'o9_34801']],
 [a^4 - a^3 + a^2 - a + 1, ['m201', 'o9_34818']],
 [a^4 - 2*a^3 + 3*a^2 - 2*a + 1,
  ['m222', 's704', 'v2543', 't07599', 'o9_20894']],
 [a^4 - a^3 - a^2 - a + 1, ['m224', 'v3093', 't11814']],
 [4*a^2 - 9*a + 4, ['s016', 's726', 'o9_34700']],
 [5*a^2 - 9*a + 5, ['s023', 'o9_31990']],
 [a^6 - a^5 + a^3 - a + 1, ['s188', 'o9_22337', 'o9_41995']],
 [a^6 - a^5 - a^3 - a + 1, ['s194', 't10500']],
 [a^4 - a^2 + 1, ['s239', 't11441']],
 [a^6 - 2*a^5 + a^3 - 2*a + 1, ['s451', 'o9_30542', 'o9_40291']],
 [a^4 + a^3 - 3*a^2 + a + 1, ['s580', 't09927']],
 [a^4 - 3*a^3 + 5*a^2 - 3*a + 1, ['s912', 'o9_40568']],
 [5*a^2 - 11*a + 5, ['v0016', 'v2257', 'o9_37795']],
 [6*a^2 - 11*a + 6, ['v0025', 'v2284', 

For each of these 74 groups we check Dunfiled's list for exceptional surgeries and if it is hyperbolic we build the 0-surgery and compare their verified volumes.

In [4]:
def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=index,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def overlap(x,y):
    '''Checks if the two intervals of x and y overlap.'''
    if y.endpoints()[0]<=x.center()<=y.endpoints()[1]:
        return True
    return False

In [7]:
exceptional_fillings=[]
with open('exceptional_fillings.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        exceptional_fillings.append(row)
len(exceptional_fillings)

205823

In [30]:
start_time = time.time()

ALEX_GROUPS_WITH_VOLUMENS=[]
ALEX_GROUPS_WITH_REGINA_NAMES=[]

for x in ALEX_GROUPS_WITH_MORE_THAN_ONE_KNOT:
    volumes=[]
    names=[]
    for name in x[1]:
        K=snappy.Manifold(name)
        zero_slope=K.homological_longitude()
        hyperbolic=True
        for x in exceptional_fillings:
            if x[-2]==name:
                if x[-1]==str(zero_slope):
                    names.append([name,x[2]])
                    hyperbolic=False
        if hyperbolic:
            K.dehn_fill(zero_slope)
            vol=better_volume(K,index=100)
            if vol==0:
                vol=better_volume(K,index=1000)
            if vol==0:
                vol=better_volume(K,index=10000,try_hard=True)
            volumes.append([name,vol])
    if len(volumes)>1:
        volumes.sort(key=lambda x: x[1])
        ALEX_GROUPS_WITH_VOLUMENS.append(volumes)
    if len(names)>1:
        ALEX_GROUPS_WITH_REGINA_NAMES.append(names)

print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

Time taken: 2.100540924270948 hours 


In [31]:
ALEX_GROUPS_WITH_VOLUMENS

[[['m053', 0], ['v3169', 0]],
 [['m199', 3.177293278601?], ['o9_34801', 3.177293278600?]],
 [['m224', 2.78183391240?],
  ['v3093', 2.781833912396?],
  ['t11814', 5.64120269706?]],
 [['o9_22337', 0], ['o9_41995', 6.53035855484?]],
 [['s194', 3.50909036378?], ['t10500', 6.577389902878?]],
 [['s451', 4.48795616567?],
  ['o9_30542', 6.33580908350?],
  ['o9_40291', 7.57772895633?]],
 [['s580', 4.408180344705?], ['t09927', 4.55591888988?]],
 [['s912', 4.059766425639?], ['o9_40568', 5.67188354357?]],
 [['v0025', 0], ['o9_33067', 0]],
 [['v0329', 3.82422316958?],
  ['o9_23786', 4.724193070534?],
  ['o9_29669', 7.010981552339?]],
 [['v0535', 3.56139987499?], ['v0960', 4.639202471219?]],
 [['v0600', 4.393635924020?], ['o9_40171', 7.85708783364?]],
 [['v1063', 4.613255212924?], ['o9_33263', 7.33009980619?]],
 [['v1935', 5.389888525936?], ['o9_38471', 7.58143920443?]],
 [['v1964', 4.686118265095?], ['o9_43369', 7.80830657390?]],
 [['t10887', 0], ['t11418', 0]],
 [['v2166', 5.38541418815?], ['o9_39

In [32]:
ALEX_GROUPS_WITH_REGINA_NAMES

[[['m004', 'T x I / [ 2,1 | 1,1 ]'],
  ['t11898',
   "JSJ([('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')])"]],
 [['m032', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
  ['s879', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]'],
  ['v2861', 'SFS [A: (2,1)] / [ 2,7 | 1,3 ]'],
  ['t08306', 'SFS [A: (2,1)] / [ 2,9 | 1,4 ]'],
  ['o9_22066', 'SFS [A: (2,1)] / [ 2,11 | 1,5 ]']],
 [['s648',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
  ['v2001', 'SFS [A: (4,1)] / [ -1,3 | 1,-2 ]'],
  ['v2743',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"]],
 [['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']],
 [['s704',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -1,2 | 0,1 ]'],
  ['v2543',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -2,3 | -1,2 ]'],
  ['t07599',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -3,4 | -2,3 ]'],
  ['o9_20894',
   'SFS [D: (2,1)

We start working with the exceptional surgeries. If we have equal gluings we can distinguish the manifolds via the trace of their gluing matrices. (Or of the traces agree we check explicitly that the matrices are not conjugated.) If the manifolds are JSJ manifolds we need to understand the gluing. Here we could try to distingusih them (or prove equality) via their fundamental groups. It might also be that some JSJ manifolds are actually SFS. So here we also need to work a bit harder.

In [57]:
equal_exceptional_surgeries=[]
equal_exceptional_surgeries.append([['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']])
equal_exceptional_surgeries.append([['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
['t11441', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']])
equal_exceptional_surgeries.append([['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']])

possible_equal_exceptional_surgeries=[]
possible_equal_exceptional_surgeries.append([['s648', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
['v2001', 'SFS [A: (4,1)] / [ -1,3 | 1,-2 ]'],
['v2743', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"]])
possible_equal_exceptional_surgeries.append([['s726', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
['o9_34700', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,3)]')])"]])
possible_equal_exceptional_surgeries.append([['v2284', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
['o9_34323', "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]])
possible_equal_exceptional_surgeries.append([['o9_35240', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"],
['o9_36459', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"]])
possible_equal_exceptional_surgeries.append([['v2362', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
['t00017', 'SFS [A: (6,1)] / [ 0,1 | 1,-2 ]'],
['t08617', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
['t11577', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
['o9_36180', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
['o9_40361', "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]])
possible_equal_exceptional_surgeries.append([['t05803', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (4,7)]')])"],
['o9_00022', 'SFS [A: (8,7)] / [ 0,1 | 1,-1 ]'],
['o9_24670', 'SFS [A: (8,3)] / [ 2,5 | 1,2 ]']])
possible_equal_exceptional_surgeries.append([['t07734', "JSJ([('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
['o9_36955', "JSJ([('SFSpace', 'SFS [A: (3,4)]'), ('SFSpace', 'SFS [A: (3,4)]')])"]])

For these remaining examples, we use fundamentalgroup or explicit Kirby calculus. (Also to check the correct Orientations.) For the fundamental group we can use regina. This is largely implemented.

We continue to work with the hyperbolic surgeries. First we remove the pairs with different volumes.

In [54]:
pairs_with_unclear_volumes=[]
pairs_with_same_volume=[]

for x in ALEX_GROUPS_WITH_VOLUMENS:
    for y in x:
        if y[1]==0:
            pairs_with_unclear_volumes.append(x)
            break
    if x not in pairs_with_unclear_volumes:
        for i in range(len(x)-1):
            three=False
            vol1=x[i][1]
            vol2=x[i+1][1]
            if overlap(vol1,vol2):
                print(x[i],x[i+1])
                try:
                    vol3=x[i+2][1]
                    if overlap(vol2,vol3):
                        print(x[i],x[i+1],x[i+2])
                        pairs_with_same_volume.append([x[i],x[i+1],x[i+2]])
                        three=True
                except:
                    pass
                if three==False:
                    pairs_with_same_volume.append([x[i],x[i+1]])

        

['m199', 3.177293278601?] ['o9_34801', 3.177293278600?]
['m224', 2.78183391240?] ['v3093', 2.781833912396?]
['v2508', 3.702897321857?] ['v3195', 3.702897321857?]
['t11532', 4.90949251141?] ['t12200', 4.90949251141?]
['t11532', 4.90949251141?] ['t12200', 4.90949251141?] ['o9_43446', 4.909492511407?]
['t12200', 4.90949251141?] ['o9_43446', 4.909492511407?]
['o9_39433', 5.59095176819?] ['o9_40519', 5.59095176819?]
['v2869', 4.97179004792?] ['t12388', 4.971790047912?]
['t07281', 4.162020103638?] ['o9_34949', 4.16202010364?]
['t07281', 4.162020103638?] ['o9_34949', 4.16202010364?] ['o9_39806', 4.162020103638?]
['o9_34949', 4.16202010364?] ['o9_39806', 4.162020103638?]
['t10974', 5.298314671809?] ['o9_39967', 5.298314671809?]
['t09735', 3.925475315569?] ['t11748', 3.92547531557?]
['t09900', 4.455978629569?] ['o9_34908', 4.45597862957?]
['t09900', 4.455978629569?] ['o9_34908', 4.45597862957?] ['o9_43876', 4.455978629569?]
['o9_34908', 4.45597862957?] ['o9_43876', 4.455978629569?]
['t11900', 6

In [55]:
pairs_with_same_volume

[[['m199', 3.177293278601?], ['o9_34801', 3.177293278600?]],
 [['m224', 2.78183391240?], ['v3093', 2.781833912396?]],
 [['v2508', 3.702897321857?], ['v3195', 3.702897321857?]],
 [['t11532', 4.90949251141?],
  ['t12200', 4.90949251141?],
  ['o9_43446', 4.909492511407?]],
 [['t12200', 4.90949251141?], ['o9_43446', 4.909492511407?]],
 [['o9_39433', 5.59095176819?], ['o9_40519', 5.59095176819?]],
 [['v2869', 4.97179004792?], ['t12388', 4.971790047912?]],
 [['t07281', 4.162020103638?],
  ['o9_34949', 4.16202010364?],
  ['o9_39806', 4.162020103638?]],
 [['o9_34949', 4.16202010364?], ['o9_39806', 4.162020103638?]],
 [['t10974', 5.298314671809?], ['o9_39967', 5.298314671809?]],
 [['t09735', 3.925475315569?], ['t11748', 3.92547531557?]],
 [['t09900', 4.455978629569?],
  ['o9_34908', 4.45597862957?],
  ['o9_43876', 4.455978629569?]],
 [['o9_34908', 4.45597862957?], ['o9_43876', 4.455978629569?]],
 [['t11900', 6.46054090363?], ['o9_40803', 6.4605409037?]],
 [['o9_33833', 4.73709732245?], ['o9_375

In [56]:
pairs_with_unclear_volumes

[[['m053', 0], ['v3169', 0]],
 [['o9_22337', 0], ['o9_41995', 6.53035855484?]],
 [['v0025', 0], ['o9_33067', 0]],
 [['t10887', 0], ['t11418', 0]],
 [['v2272', 0],
  ['v3423', 4.415332477454?],
  ['v3536', 4.41533247746?],
  ['o9_42735', 4.415332477454?],
  ['t12607', 5.28793627053?],
  ['t12748', 5.28793627053?],
  ['o9_41909', 5.83941124979?],
  ['o9_42515', 5.83941124979?]],
 [['t00027', 0], ['o9_39339', 0]],
 [['t05252', 0], ['o9_19326', 0]],
 [['o9_10654', 0], ['o9_19831', 0]],
 [['o9_31828', 0], ['o9_37768', 0]]]

We further try to distinguish pairs via their symmetry groups:

In [73]:
more=[]
for x in pairs_with_unclear_volumes:
    if len(x)>2:
        more=x
        
    else:
        K1=snappy.Manifold(x[0][0])
        K2=snappy.Manifold(x[1][0])
        K1.dehn_fill(K1.homological_longitude())
        K2.dehn_fill(K1.homological_longitude())
        print(x)
        print(K1.symmetry_group())
        print(K2.symmetry_group())
        print('-------------')

[['m053', 0], ['v3169', 0]]
at least Z/2 + Z/2
Z/2
-------------
[['o9_22337', 0], ['o9_41995', 6.53035855484?]]
at least Z/2
Z/2
-------------
[['v0025', 0], ['o9_33067', 0]]
at least Z/2 + Z/2
Z/2
-------------
[['t10887', 0], ['t11418', 0]]
at least Z/2
Z/2
-------------
[['t00027', 0], ['o9_39339', 0]]
at least Z/2 + Z/2
D6
-------------
[['t05252', 0], ['o9_19326', 0]]
at least Z/2 + Z/2
at least Z/2 + Z/2
-------------
[['o9_10654', 0], ['o9_19831', 0]]
at least Z/2 + Z/2
at least Z/2 + Z/2
-------------
[['o9_31828', 0], ['o9_37768', 0]]
at least Z/2
at least Z/2
-------------


In [17]:
more=['v2272', 'v3423', 'v3536', 'o9_42735', 't12607', 't12748', 'o9_41909', 'o9_42515']

In [20]:
for x in more:
    print(x)
    K=snappy.Manifold(x)
    K.dehn_fill(K.homological_longitude())
    print(K)
    print(K.symmetry_group())


v2272
v2272(2,-1)
at least Z/2
v3423
v3423(2,-1)
Z/2 + Z/2
v3536
v3536(2,1)
Z/2 + Z/2
o9_42735
o9_42735(1,1)
Z/2 + Z/2
t12607
t12607(2,1)
Z/2
t12748
t12748(2,-1)
Z/2
o9_41909
o9_41909(2,-1)
Z/2
o9_42515
o9_42515(2,1)
Z/2


The problem is that we do not find a positive triangulation for the pairs where the volume does not work. We need to check again how Dunfield did it. We are left with the following pairs which might be the same. And we can probably show that they are not when we find a positive triangulation.

In [41]:
might_be_the_same=[['o9_22337', 'o9_41995'],
['t10887', 't11418'],
['t00027', 'o9_39339'],
['t05252', 'o9_19326'],
['o9_10654', 'o9_19831'],
['o9_31828', 'o9_37768'],
['v2272','v3423','v3536', 'o9_42735'],
['v2272','t12607','t12748','o9_41909','o9_42515']]

In [37]:
pairs_with_same_volume=[['m199', 'o9_34801'],
 ['m224', 'v3093'],
 ['v2508', 'v3195'],
 ['t11532', 't12200', 'o9_43446'],
 ['t12200', 'o9_43446'],
 ['o9_39433','o9_40519'],
 ['v2869', 't12388'],
 ['t07281', 'o9_34949', 'o9_39806'],
 ['o9_34949', 'o9_39806'],
 ['t10974', 'o9_39967'],
 ['t09735', 't11748'],
 ['t09900', 'o9_34908', 'o9_43876'],
 ['o9_34908', 'o9_43876'],
 ['t11900', 'o9_40803'],
 ['o9_33833', 'o9_37547']]

In [38]:
for y in pairs_with_same_volume:
    for x in y:
        K=snappy.Manifold(x)
        K.dehn_fill(K.homological_longitude())
        print(K)
        print(K.symmetry_group())
    print('-------------')

m199(4,-1)
Z/2 + Z/2
o9_34801(1,-1)
Z/2 + Z/2
-------------
m224(3,-1)
Z/2 + Z/2
v3093(1,-1)
Z/2 + Z/2
-------------
v2508(2,-1)
Z/2
v3195(2,-1)
Z/2
-------------
t11532(2,1)
0
t12200(2,-1)
0
o9_43446(1,-1)
0
-------------
t12200(2,-1)
0
o9_43446(1,-1)
0
-------------
o9_39433(2,1)
0
o9_40519(2,-1)
0
-------------
v2869(3,-1)
Z/2
t12388(2,1)
Z/2
-------------
t07281(3,-1)
Z/2 + Z/2
o9_34949(1,1)
Z/2 + Z/2
o9_39806(1,1)
Z/2 + Z/2
-------------
o9_34949(1,1)
Z/2 + Z/2
o9_39806(1,1)
Z/2 + Z/2
-------------
t10974(3,-1)
Z/2
o9_39967(2,-1)
Z/2
-------------
t09735(1,-1)
Z/2 + Z/2
t11748(1,1)
Z/2 + Z/2
-------------
t09900(2,1)
Z/2 + Z/2
o9_34908(1,-1)
Z/2 + Z/2
o9_43876(1,1)
Z/2 + Z/2
-------------
o9_34908(1,-1)
Z/2 + Z/2
o9_43876(1,1)
Z/2 + Z/2
-------------
t11900(4,1)
0
o9_40803(3,1)
0
-------------
o9_33833(1,1)
Z/2 + Z/2
o9_37547(1,-1)
Z/2 + Z/2
-------------


So they have all the same symmetry groups. We check the length spectrum:

In [40]:
for y in pairs_with_same_volume:
    for x in y:
        K=snappy.Manifold(x)
        K.dehn_fill(K.homological_longitude())
        print(K)
        print(K.length_spectrum(full_rigor=True))
    print('-------------')

m199(4,-1)
mult length                           topology      parity
2    0.561218218060211 + 1.23214410440380*I circle        orientation-preserving
1    0.609377863436009 - 2.73796784112515*I circle        orientation-preserving
o9_34801(1,-1)
mult length                           topology      parity
2    0.561218218060192 - 1.23214410440408*I circle        orientation-preserving
1    0.609377863436000 + 2.73796784112515*I circle        orientation-preserving
-------------
m224(3,-1)
mult length                           topology      parity
1    0.252670219220850 + 2.19726867750004*I circle        orientation-preserving
2    0.853210316468308 + 1.57944204815431*I circle        orientation-preserving
v3093(1,-1)
mult length                           topology      parity
1    0.252670219229678 - 2.19726867750490*I circle        orientation-preserving
2    0.853210316467587 - 1.57944204815586*I circle        orientation-preserving
-------------
v2508(2,-1)
mult length                

o9_40803(3,1)
mult length                           topology      parity
1    0.515258462947825 + 1.38531790328723*I circle        orientation-preserving
1    0.537327915109287 + 2.26137375017886*I circle        orientation-preserving
-------------
o9_33833(1,1)
mult length                           topology      parity
1    0.289839709193995 - 1.96081354896931*I circle        orientation-preserving
1    0.378504828950961 + 2.22052391894670*I circle        orientation-preserving
1    0.901475419444761 + 1.24522212215954*I circle        orientation-preserving
o9_37547(1,-1)
mult length                           topology      parity
1    0.289839709193527 - 1.96081354896970*I circle        orientation-preserving
1    0.378504828950961 + 2.22052391894670*I circle        orientation-preserving
1    0.901475419444761 + 1.24522212215951*I circle        orientation-preserving
-------------


So it seems that these are all comensurable, so the best way to distinguish them is via the fundamental group. 

Before we do that we actually search for an isometry:

In [47]:
for y in pairs_with_same_volume:
    print(y)
    for x in y:
        K=snappy.Manifold(x)
        K.dehn_fill(K.homological_longitude())
        print(K.identify())
    print('------')

['m199', 'o9_34801']
[m199(-4,1)]
[m199(-4,1)]
------
['m224', 'v3093']
[]
[]
------
['v2508', 'v3195']
[m345(1,2)]
[m345(1,2)]
------
['t11532', 't12200', 'o9_43446']
[v2986(1,2)]
[v2986(1,2)]
[v2986(1,2)]
------
['t12200', 'o9_43446']
[v2986(1,2)]
[v2986(1,2)]
------
['o9_39433', 'o9_40519']
[]
[]
------
['v2869', 't12388']
[v2209(2,3)]
[v2209(2,3)]
------
['t07281', 'o9_34949', 'o9_39806']
[]
[]
[]
------
['o9_34949', 'o9_39806']
[]
[]
------
['t10974', 'o9_39967']
[s928(2,3)]
[s928(2,3)]
------
['t09735', 't11748']
[]
[]
------
['t09900', 'o9_34908', 'o9_43876']
[s528(-1,3)]
[s528(-1,3)]
[s528(-1,3)]
------
['o9_34908', 'o9_43876']
[s528(-1,3)]
[s528(-1,3)]
------
['t11900', 'o9_40803']
[]
[]
------
['o9_33833', 'o9_37547']
[]
[]
------


In [51]:
might_be_equal_with_same_volume=[['m224', 'v3093'],['o9_39433', 'o9_40519'],['t07281', 'o9_34949', 'o9_39806'],
['t09735', 't11748'],['t11900', 'o9_40803'],['o9_33833', 'o9_37547']]

In [54]:
for y in might_be_equal_with_same_volume:
    print(y)
    if len(y)==3:
        K1=snappy.Manifold(y[0])
        K1.dehn_fill(K1.homological_longitude())
        K2=snappy.Manifold(y[1])
        K2.dehn_fill(K2.homological_longitude())
        K3=snappy.Manifold(y[2])
        K3.dehn_fill(K3.homological_longitude())
        print(K1.is_isometric_to(K2))
        print(K1.is_isometric_to(K3))
        print(K2.is_isometric_to(K3))
    else:
        K1=snappy.Manifold(y[0])
        K1.dehn_fill(K1.homological_longitude())
        K2=snappy.Manifold(y[1])
        K2.dehn_fill(K2.homological_longitude())
        print(K1.is_isometric_to(K2))
    print('------') 

['m224', 'v3093']
True
------
['o9_39433', 'o9_40519']
True
------
['t07281', 'o9_34949', 'o9_39806']
True
True
True
------
['t09735', 't11748']
True
------
['t11900', 'o9_40803']
True
------
['o9_33833', 'o9_37547']
True
------


In [55]:
equal_hyperbolic_surgeries=[['m199', 'o9_34801'],['v2508', 'v3195'],['t11532', 't12200', 'o9_43446'],
['v2869', 't12388'],['t10974', 'o9_39967'],['t09900', 'o9_34908', 'o9_43876'],['m224', 'v3093'],['o9_39433', 'o9_40519'],['t07281', 'o9_34949', 'o9_39806'],
['t09735', 't11748'],['t11900', 'o9_40803'],['o9_33833', 'o9_37547']]

So they are all equal.

To summarize we found the following:

(1) A list called 'equal_hyperbolic_surgeries' of knots that share the same hyperbolic 0-surgery.

(2) A list called 'equal_exceptional_surgeries' of knots that share the same exceptional 0-surgery.

(3) A list called 'might_be_the_same' of knots that might have the same hyperbolic 0-surgery.

(4) A list called 'possible_equal_exceptional_surgeries' of knots that might have the same exceptional 0-surgery.

These lists are below. In case (3) and (4) we can try to use regina. (Either use fundamental group or search directly for a combinatorial isomerphism.)


In [59]:
equal_hyperbolic_surgeries

[['m199', 'o9_34801'],
 ['v2508', 'v3195'],
 ['t11532', 't12200', 'o9_43446'],
 ['v2869', 't12388'],
 ['t10974', 'o9_39967'],
 ['t09900', 'o9_34908', 'o9_43876'],
 ['m224', 'v3093'],
 ['o9_39433', 'o9_40519'],
 ['t07281', 'o9_34949', 'o9_39806'],
 ['t09735', 't11748'],
 ['t11900', 'o9_40803'],
 ['o9_33833', 'o9_37547']]

In [60]:
equal_exceptional_surgeries

[[['m201', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
  ['o9_34818', 'SFS [S2: (2,1) (5,1) (10,-7)]']],
 [['s239', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
  ['t11441',
   'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']],
 [['v0595', 'SFS [S2: (2,1) (7,2) (14,-11)]'],
  ['t12120', 'SFS [S2: (2,1) (7,2) (14,-11)]']]]

In [61]:
might_be_the_same

[['o9_22337', 'o9_41995'],
 ['t10887', 't11418'],
 ['t00027', 'o9_39339'],
 ['t05252', 'o9_19326'],
 ['o9_10654', 'o9_19831'],
 ['o9_31828', 'o9_37768'],
 ['v2272', 'v3423', 'v3536', 'o9_42735'],
 ['v2272', 't12607', 't12748', 'o9_41909', 'o9_42515']]

In [62]:
possible_equal_exceptional_surgeries

[[['s648',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
  ['v2001', 'SFS [A: (4,1)] / [ -1,3 | 1,-2 ]'],
  ['v2743',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"]],
 [['s726',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
  ['o9_34700',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,3)]')])"]],
 [['v2284',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,2)]')])"],
  ['o9_34323',
   "JSJ([('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,5)]')])"]],
 [['o9_35240', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"],
  ['o9_36459', "JSJ([('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm043')])"]],
 [['v2362',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
  ['t00017', 'SFS [A: (6,1)] / [ 0,1 | 1,-2 ]'],
  ['t08617',
   "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,1)]')])"],
  ['t11577',
   "JSJ([('SFSpace', 'SFS [A: (2